# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [442]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
import patsy
import statsmodels.formula.api as smf

In [443]:
previsao_renda_df = pd.read_csv('previsao_de_renda.csv')
previsao_renda_df.sample(5)


,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
8973,8973,2015-09-01,8481,F,True,True,1,Empresário,Secundário,Casado,Casa,37,8.893151,3.0,1763.92
823,823,2015-01-01,6683,F,True,False,1,Empresário,Superior completo,Casado,Casa,39,6.898630,3.0,9963.64
8794,8794,2015-09-01,15198,M,False,True,1,Empresário,Secundário,União,Casa,46,14.375342,3.0,16102.27
13586,13586,2016-02-01,10918,F,False,False,0,Assalariado,Secundário,União,Governamental,38,3.849315,2.0,4059.58
14655,14655,2016-03-01,14469,F,False,True,0,Pensionista,Secundário,Casado,Casa,63,NaN,2.0,3436.50


In [444]:
previsao_renda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [445]:
previsao_renda_df.drop(columns=['Unnamed: 0'], inplace=True)
previsao_renda_df.drop(columns=['data_ref'], inplace=True)
previsao_renda_df.drop(columns=['id_cliente'], inplace=True)

In [446]:
previsao_renda_df.columns

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda'],
      dtype='object')

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [447]:
# 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

df_train, df_test = train_test_split(previsao_renda_df, test_size=0.25, random_state=42)

regularization_alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

modelo = """renda ~ sexo 
        + posse_de_veiculo 
        + posse_de_imovel 
        + qtd_filhos 
        + tipo_renda 
        + educacao 
        + estado_civil 
        + tipo_residencia 
        + idade 
        + tempo_emprego 
        + qt_pessoas_residencia
    """

# Transformo ara matriz design

y_train, X_train = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=modelo, data=df_test)

In [448]:
# 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_2y_pred = []

for alpha in alphas:
   md = sm.OLS(y_train, X_train)
   reg = md.fit_regularized(method='elastic_net', 
                            refit=True, 
                            L1_wt=0,  # ridge fit
                            alpha=alpha)
   y_pred = reg.predict(X_test)
   r2 = r2_score(y_test, y_pred)
   ridge_2y_pred.append(r2)
   print(f'Alpha {alpha}: \nR-squared = {r2}\n')

pd.DataFrame({'alpha':alphas, 
             '𝑅2':ridge_2y_pred
            }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.26243403442818525

Alpha 0.001: 
R-squared = 0.2622619495235067

Alpha 0.005: 
R-squared = 0.2619663776008285

Alpha 0.01: 
R-squared = 0.26156847585828613

Alpha 0.05: 
R-squared = 0.2563827760732651

Alpha 0.1: 
R-squared = 0.2486340863705101



,alpha,𝑅2
0,0.000,0.262434
1,0.001,0.262262
2,0.005,0.261966
3,0.010,0.261568
4,0.050,0.256383
5,0.100,0.248634


In [449]:
# 3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

lasso_2y_pred = []

for alpha in alphas:
   md = sm.OLS(y_train, X_train)
   reg = md.fit_regularized(method='elastic_net', 
                            refit=True, 
                            L1_wt=1,  # lasso fit
                            alpha=alpha)
   y_pred = reg.predict(X_test)
   r2 = r2_score(y_test, y_pred)
   lasso_2y_pred.append(r2)
   print(f'Alpha {alpha}: \nR-squared = {r2}\n')

pd.DataFrame({'alpha':alphas, 
             '𝑅2':lasso_2y_pred
            }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.26243403442818525

Alpha 0.001: 
R-squared = 0.26243403442818525

Alpha 0.005: 
R-squared = 0.26243403442818525

Alpha 0.01: 
R-squared = 0.26243403442818525

Alpha 0.05: 
R-squared = 0.26250010213196595

Alpha 0.1: 
R-squared = 0.26243403442818525



,alpha,𝑅2
4,0.050,0.262500
0,0.000,0.262434
1,0.001,0.262434
2,0.005,0.262434
3,0.010,0.262434
5,0.100,0.262434


O Ridge tem o maior valor com o Alpha 0.000 e R-quadrado 0.262434, enquanto Lasso tem o maior valor com o Alpha 0.050 e R-quadrado 0.262500. O Lasso apresentou o maior R-quadrado na comparação.

In [450]:
# 4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
stepwise_x_teste = df_test.copy()

stepwise_x_teste.dropna(inplace=True)
object_columns = []
for column in stepwise_x_teste.columns:
    if stepwise_x_teste[column].dtype not in [int, bool, float]:
        object_columns.append(column)

stepwise_x_teste = pd.get_dummies(
    stepwise_x_teste,
    columns=object_columns,
    drop_first=True
)

for column in stepwise_x_teste.columns:
    if stepwise_x_teste[column].dtype == bool:
        stepwise_x_teste[column] = stepwise_x_teste[column].astype(int)

X = stepwise_x_teste.drop(columns=["renda"]).copy()
y = stepwise_x_teste["renda"]


def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 3.04548e-111
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.83858e-94
#############
['tempo_emprego', 'sexo_M']
Add  idade                          with p-value 0.00110246
#############
['tempo_emprego', 'sexo_M', 'idade']
Add  posse_de_imovel                with p-value 0.0128449
#############
['tempo_emprego', 'sexo_M', 'idade', 'posse_de_imovel']
Add  educacao_Superior completo     with p-value 0.0192688
#############
['tempo_emprego', 'sexo_M', 'idade', 'posse_de_imovel', 'educacao_Superior completo']
#############
['tempo_emprego', 'sexo_M', 'idade', 'posse_de_imovel', 'educacao_Superior completo']
resulting features:
['tempo_emprego', 'sexo_M', 'idade', 'posse_de_imovel', 'educacao_Superior completo']


5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

O melhor resultado foi p-value 0.00110246 com a combinação ['tempo_emprego', 'sexo_M']

In [451]:
df_test.columns
df_test["qtd_filhos"].value_counts()

qtd_filhos
0     2589
1      752
2      360
3       42
4        6
14       1
Name: count, dtype: int64

In [459]:
# 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
reg = smf.ols('np.log(renda) ~ tempo_emprego + sexo + posse_de_imovel + idade', data = df_test).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     563.3
Date:                Mon, 13 May 2024   Prob (F-statistic):          2.33e-292
Time:                        15:20:51   Log-Likelihood:                -3409.3
No. Observations:                3114   AIC:                             6827.
Df Residuals:                    3110   BIC:                             6851.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   7.4340      0.030    251.203      0.000       7.376       7.492
sexo[T.M]                   0.7917      0.027     29.198      0.000       0.739       0.845
posse_de_imovel[T.True]     0.1096      0.027      3.998      0.000       0.056       0.163
tempo_emprego               0.0647      0.002     32.474      0.000       0.061       0.069
==============================================================================
Omnibus:                        5.197   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.074   Jarque-Bera (JB):                5.929
Skew:                          -0.008   Prob(JB):                       0.0516
Kurtosis:                       3.213   Cond. No.                         29.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [462]:
# 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

regr_1 = DecisionTreeRegressor(max_depth=3)
regr_1.fit(X_test, y_test)
y_pred_1 = regr_1.predict(X_test)
r2_test_1 = r2_score(y_test, y_pred_1)

print("R-quadrado para o modelo com max_depth=2:", r2_test_1)

R-quadrado para o modelo com max_depth=2: 0.40610790039116373
